In [1]:
# %pip install neupy

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import warnings; warnings.simplefilter('ignore')

In [4]:
import numpy as np
import pandas as pd
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from neupy import algorithms
from sklearn import metrics
from math import sqrt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from multiprocessing.pool import ThreadPool
from prettytable import PrettyTable

import statsmodels.api as sm

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Pre-process the data

In [5]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

# Removing columns based on the dependent column

In [6]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

# Feature Selection Methods

In [7]:
def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]]).astype(float)).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

In [8]:
def backward_elimination(data, target,significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features]).astype(float)
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

# GNN and its best parameters function

In [9]:
def bestParameters(X,Y,std = 0.1):
    model = RandomizedSearchCV(algorithms.GRNN(std=std, verbose=False), param_distributions={'std': np.arange(1e-2, 1, 1e-3)}, scoring='neg_mean_squared_error',)
    X_train, X_test, y_train, y_test = train_test_split(preprocessing.minmax_scale(X), preprocessing.minmax_scale(Y), test_size=0.3, random_state=0)
    model.fit(X_train,y_train)
    return model.best_params_

In [10]:
# def generalized_neural_network(X,Y,std=0.1):
#     best = bestParameters(X,Y,std)
#     x_train, x_test, y_train, y_test = train_test_split(preprocessing.minmax_scale(X), preprocessing.minmax_scale(Y), test_size=0.3, random_state=0)
#     model = algorithms.GRNN(std=best['std'])
#     model.train(x_train,y_train)
#     y_pred = model.predict(x_test)
#     rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
#     mae = metrics.mean_absolute_error(y_test, y_pred)
#     mse = metrics.mean_squared_error(y_test, y_pred)
#     r2 = metrics.r2_score(y_test, y_pred)
#     c = 0
#     for a,b in zip(y_test, y_pred):
#         if a*b >= 0:
#             c += 1
#     direction = c/len(y_test)
#     myres =  {"RMSE":rmse,"MAE":mae,"MSE":mse,"rsquared_adj":r2,"std":std, "best-std":best['std'],"Percentage":direction}
#     print("done")
#     return myres

In [11]:
def generalized_neural_network(X,Y,std=0.1):
    best = bestParameters(X,Y,std)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    model = algorithms.GRNN(std=best['std'])
    model.train(x_train,y_train)
    y_pred = model.predict(x_test)
    
#     print(list(zip(y_test, y_pred)))
    
    y_pred = np.nan_to_num(y_pred)
    
    rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)
    c = 0
    for a,b in zip(y_test, y_pred):
        if a*b >= 0:
            c += 1
    direction = c/len(y_test)
    myres =  {"RMSE":rmse,"MAE":mae,"MSE":mse,"rsquared_adj":r2,"std":std, "best-std":best['std'],"Percentage":direction}
#     print("done")
    return myres

# Finding results from each set of important features

In [12]:
def fit_GNN(df, column, method, name, results):
    print("GNN Model fitted using columns obtained from feature importance using " + method + " : ")
    X = df[df.columns[:-1]]
    Y = df[column].values
    
    data = list()
    stds = np.linspace(0,1,11)
    arguments = list()
    max_percentage = 0
    for s in stds:
        data = [X,Y,s]
        arguments.append(data)
        model_result = generalized_neural_network(X, Y, s)
        if (max_percentage < model_result["Percentage"]):
            max_percentage = model_result["Percentage"]
#     threads = ThreadPool(4)
#     model_result = threads.starmap(generalized_neural_network,arguments)
    
#         print(model_result)
    
#         print("Percentage of correct direction : ", model_result["Percentage"])
        
    
        results["GNN_Regression_FI_" + method + "_stds_" + str(round(s, 2))] = model_result["Percentage"]
    
        create_pretty_table(name , "GNN_Regression", method + "-" + str(round(s, 2)), model_result)
    print("Maximum percentage of correct direction : ", max_percentage)

In [13]:
def get_results_from_FI_ForwardSelection(df1, name, column, results):
    print("Features Importance using Forward Selection Method")
    print("*****************************************************************************************")
    method = "ForwardSelection"
    X = df1[df1.columns[:-1]]
    Y = df1[column].values
    forward_features = forward_selection(X,Y)
    print("Features obtained from Forward Selection method : ") 
    print("--------------------------------------")
    print(forward_features)
    if (len(forward_features) != 0):
        forward_features.append(column)
        df_fs = df1[forward_features]
        fit_GNN(df_fs, column, method, name, results)
    print("*****************************************************************************************")

In [14]:
def get_results_from_FI_BackwardElimination(df1, name, column, results):
    print("Features Importance using Backward Elimination Method")
    print("*****************************************************************************************")
    method = "BackwardElimination"
    X = df1[df1.columns[:-1]]
    Y = df1[column].values
    backward_features = backward_elimination(X,Y)
    print("Features obtained from Backward Elimination method : ") 
    print("--------------------------------------")
    print(backward_features)
    if (len(backward_features) != 0):
        backward_features.append(column)
        df_be = df1[backward_features]
        fit_GNN(df_be, column, method, name, results)
    print("*****************************************************************************************")

In [15]:
def get_results_from_FI_AllFeatures(df1, name, column, results):
    print("All Features are considered : ")
    print("*****************************************************************************************")
    method = "AllFeaturesConsideration"
    X = df1[df1.columns[:-1]]
    Y = df1[column].values
    all_features = list(X.columns)
    print("All Features are --->>") 
    print("--------------------------------------")
    print(all_features)
    if (len(all_features) != 0):
        all_features.append(column)
        df_all = df1[all_features]
        fit_GNN(df_all, column, method, name, results)
    print("*****************************************************************************************")

In [16]:
def get_results_from_each_set(data, name, final_df):
    df = pre_process_data(data, 60)
    column = "Next Day Close Price GR"
    (df1, column) = dependent_column(df, column)
    results = {}
    get_results_from_FI_ForwardSelection(df1, name, column, results)
    get_results_from_FI_BackwardElimination(df1, name, column, results)
    get_results_from_FI_AllFeatures(df1, name, column, results)
    sorted_results = sorted(results.items(), key=lambda item: item[1])
    max_row = {'Company' : name[2 : 8] + "-" + companies[name[2 : 8]], 'Model' : 'GNN-Regression', 'Method' : sorted_results[-1][0], 'Percentage' : sorted_results[-1][1]}
    final_df = final_df.append(max_row, ignore_index = True)
    print("Maximum correct direction values are obtained for {} with a percentage of {}.".format(sorted_results[-1][0], sorted_results[-1][1]))
    return final_df

# Process of getting results

In [17]:
def create_pretty_table(name, model, method, result):
    values = [name[2 : 8 ] + "-" + companies[name[2 : 8]], method] + [round(v, 6) if (not isinstance(v, str)) else v for k,v in result.items()]
    tables[model].add_row(values)
    tables[model].title = model

In [18]:
columns =['Company','Method', 'RMSE','MAE','MSE','rsquared_adj','std','best-std','Percentage']

In [19]:
companies = {"500112" : "SBIN" ,
"500325" : "RELIANCE INDUSTRIES LTD",
"532540" : "TATA CONSULTANCY SERVICES LTD" ,
"500209" : "INFOSYS LTD", 
"532174" : "ICICI BANK LTD", 
"507685" : "WIPRO LTD", 
"530965" : "INDIAN OIL CORPORATION LTD", 
"500182" : "HERO MOTOCORP LTD", 
"532210" : "CITY UNION BANK LTD", 
"500180" : "HDFC Bank Ltd",
"500680" : "PFIZER LTD", 
"506395" : "COROMANDEL iNTERNATIONAL LTD",
"500770" : "TATA CHEMICALS LTD", 
"500085" : "CHAMBAL FERTILISERS & CHEMICALS LTD", 
"501425" : "BOMBAY BURMAH TRADING CORP.LTD", 
"532899" : "KAVERI SEED COMPANY LTD", 
"537291" : "NATH BIO-GENES (INDIA) LTD", 
"500790" : "NESTLE INDIA LTD", 
"500825" : "BRITANNIA INDUSTRIES LTD", 
"533155" : "JUBILANT FOODWORKS LTD", 
"533287" : "ZEE LEARN LTD", 
"533260" : "CAREER POINT LTD", 
"539921" : "SHANTI EDUCATIONAL INITIATIVES LTD", 
"542602" : "EMBASSY OFFICE PARKS REIT", 
"543217" : "MINDSPACE BUSINESS PARKS REIT", 
"543261" : "BROOKFIELD INDIA REAL ESTATE TRUST REIT", 
"532538" : "ULTRATECH CEMENT LTD", 
"500387" : "SHREE CEMENT LTD", 
"500425" : "AMBUJA CEMENTS LTD", 
"532689" : "PVR LTD", 
"532706" : "INOX LEISURE LTD", 
"532163" : "SAREGAMA INDIA LTD", 
"524715" : "SUN PHARMACEUTICAL INDUSTRIES LTD", 
"532488" : "DIVI'S LABORATORIES LTD",
"500124" : "DR.REDDY'S LABORATORIES LTD"}

In [20]:
models = ["GNN_Regression"]
tables = {model:PrettyTable() for model in models}
for name,table in tables.items():
    table.field_names = columns

In [21]:
final_df = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Final_Results_df.csv")
# final_df.drop('Unnamed: 0', inplace = True, axis = 'columns')
final_df

,Company,Model,Method,Percentage
0,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,KNN-Regression,KNN_Regression_FI_BackwardElimination_,0.540342
1,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,Ridge Regression,RidgeFIFValue1,0.506112
2,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,Linear Regression,LinearFIBackwardElimination,0.501222
3,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,Lasso Regression,LassoFIFValue10,0.497555
4,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,Elastic Net Regression,ElasticFIFValue10,0.497555
...,...,...,...,...
187,542602-EMBASSY OFFICE PARKS REIT,Elastic Net Regression,ElasticFIPValue0.1,0.569231
188,542602-EMBASSY OFFICE PARKS REIT,Linear Regression,LinearFICoefficients0.1,0.538462
189,542602-EMBASSY OFFICE PARKS REIT,Ridge Regression,RidgeFICoefficients0.1,0.538462
190,542602-EMBASSY OFFICE PARKS REIT,KNN-Regression,KNN_Regression_FI_AllFeaturesConsideration_,0.530769


In [22]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

In [23]:
%%time
for filename in os.listdir(os.path.join(path,"Data/Stock")):
    if filename.startswith("gr"):
        df_gnn = pd.read_csv(os.path.join(path,"Data\Stock\\" + filename))
        name = os.path.join(path, "Data\Stock\\" + filename).split("\\")[-1]
        stock = name[2 : 8]
        fd_df = pd.DataFrame(columns = final_df.columns)
        print("For stock : ", stock)
        print("#################################################################################################################")
        f_df = get_results_from_each_set(df_gnn, name, fd_df)
        final_df = final_df.append(f_df, ignore_index = True)
        print("#################################################################################################################")
#         break
        
final_df = final_df.sort_values(by = ['Company', 'Percentage'], ascending = [True, False])
final_df.to_csv('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Final_Results_df.csv', index = False)

For stock :  500085
#################################################################################################################
Features Importance using Forward Selection Method
*****************************************************************************************
Features obtained from Forward Selection method : 
--------------------------------------
['Beta GR']
GNN Model fitted using columns obtained from feature importance using ForwardSelection : 
Maximum percentage of correct direction :  0.5134474327628362
*****************************************************************************************
Features Importance using Backward Elimination Method
*****************************************************************************************
Features obtained from Backward Elimination method : 
--------------------------------------
['Beta GR']
GNN Model fitted using columns obtained from feature importance using BackwardElimination : 
Maximum percentage of correct direction

Maximum percentage of correct direction :  0.5050167224080268
*****************************************************************************************
Maximum correct direction values are obtained for GNN_Regression_FI_ForwardSelection_stds_0.8 with a percentage of 0.5317725752508361.
#################################################################################################################
For stock :  500182
#################################################################################################################
Features Importance using Forward Selection Method
*****************************************************************************************
Features obtained from Forward Selection method : 
--------------------------------------
['Net Profit GR', 'Alpha GR', 'No. of Trades GR']
GNN Model fitted using columns obtained from feature importance using ForwardSelection : 
Maximum percentage of correct direction :  0.5288888888888889
********************************

Maximum percentage of correct direction :  0.5237154150197628
*****************************************************************************************
Maximum correct direction values are obtained for GNN_Regression_FI_AllFeaturesConsideration_stds_0.9 with a percentage of 0.5237154150197628.
#################################################################################################################
For stock :  500680
#################################################################################################################
Features Importance using Forward Selection Method
*****************************************************************************************
Features obtained from Forward Selection method : 
--------------------------------------
['Low Price GR', 'No. of Trades GR']
GNN Model fitted using columns obtained from feature importance using ForwardSelection : 
Maximum percentage of correct direction :  0.5306818181818181
*************************************

Maximum percentage of correct direction :  0.4912751677852349
*****************************************************************************************
All Features are considered : 
*****************************************************************************************
All Features are --->>
--------------------------------------
['Open Price GR', 'High Price GR', 'Low Price GR', 'Close Price GR', 'WAP GR', 'No.of Shares GR', 'No. of Trades GR', 'Total Turnover (Rs.) GR', 'Deliverable Quantity GR', '% Deli. Qty to Traded Qty GR', 'Spread High-Low GR', 'Spread Close-Open GR', 'Alpha GR', 'Beta GR', 'Revenue GR', 'Dividend Value GR', 'Income GR', 'Expenditure GR', 'Net Profit GR', 'EPS GR']
GNN Model fitted using columns obtained from feature importance using AllFeaturesConsideration : 
Maximum percentage of correct direction :  0.5302013422818792
*****************************************************************************************
Maximum correct direction values are obtained for

Features Importance using Forward Selection Method
*****************************************************************************************
Features obtained from Forward Selection method : 
--------------------------------------
['Beta GR', 'WAP GR', 'Alpha GR', 'Close Price GR', 'Open Price GR']
GNN Model fitted using columns obtained from feature importance using ForwardSelection : 
Maximum percentage of correct direction :  0.49653579676674364
*****************************************************************************************
Features Importance using Backward Elimination Method
*****************************************************************************************
Features obtained from Backward Elimination method : 
--------------------------------------
['Open Price GR', 'Close Price GR', 'Alpha GR', 'Beta GR']
GNN Model fitted using columns obtained from feature importance using BackwardElimination : 
Maximum percentage of correct direction :  0.5011547344110855
******

Maximum percentage of correct direction :  0.47216035634743875
*****************************************************************************************
Maximum correct direction values are obtained for GNN_Regression_FI_BackwardElimination_stds_0.5 with a percentage of 0.5100222717149221.
#################################################################################################################
For stock :  532210
#################################################################################################################
Features Importance using Forward Selection Method
*****************************************************************************************
Features obtained from Forward Selection method : 
--------------------------------------
['No. of Trades GR', 'Open Price GR']
GNN Model fitted using columns obtained from feature importance using ForwardSelection : 
Maximum percentage of correct direction :  0.5311059907834101
****************************************

Maximum percentage of correct direction :  0.5108173076923077
*****************************************************************************************
Maximum correct direction values are obtained for GNN_Regression_FI_AllFeaturesConsideration_stds_1.0 with a percentage of 0.5108173076923077.
#################################################################################################################
For stock :  532689
#################################################################################################################
Features Importance using Forward Selection Method
*****************************************************************************************
Features obtained from Forward Selection method : 
--------------------------------------
['Beta GR', 'Revenue GR']
GNN Model fitted using columns obtained from feature importance using ForwardSelection : 
Maximum percentage of correct direction :  0.5520231213872833
************************************************

Maximum percentage of correct direction :  0.4803370786516854
*****************************************************************************************
All Features are considered : 
*****************************************************************************************
All Features are --->>
--------------------------------------
['Open Price GR', 'High Price GR', 'Low Price GR', 'Close Price GR', 'WAP GR', 'No.of Shares GR', 'No. of Trades GR', 'Total Turnover (Rs.) GR', 'Deliverable Quantity GR', '% Deli. Qty to Traded Qty GR', 'Spread High-Low GR', 'Spread Close-Open GR', 'Alpha GR', 'Beta GR', 'Revenue GR', 'Dividend Value GR', 'Income GR', 'Expenditure GR', 'Net Profit GR', 'EPS GR']
GNN Model fitted using columns obtained from feature importance using AllFeaturesConsideration : 
Maximum percentage of correct direction :  0.5224719101123596
*****************************************************************************************
Maximum correct direction values are obtained for

Features obtained from Forward Selection method : 
--------------------------------------
['Close Price GR', 'Deliverable Quantity GR', 'WAP GR', 'Low Price GR', 'Spread High-Low GR', 'No. of Trades GR']
GNN Model fitted using columns obtained from feature importance using ForwardSelection : 
Maximum percentage of correct direction :  0.5384615384615384
*****************************************************************************************
Features Importance using Backward Elimination Method
*****************************************************************************************
Features obtained from Backward Elimination method : 
--------------------------------------
['Open Price GR', 'High Price GR', 'WAP GR', 'No.of Shares GR', 'No. of Trades GR', 'Total Turnover (Rs.) GR', 'Deliverable Quantity GR', '% Deli. Qty to Traded Qty GR', 'Spread High-Low GR', 'Spread Close-Open GR', 'Dividend Value GR']
GNN Model fitted using columns obtained from feature importance using BackwardEl

In [24]:
# %%time
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         filepath = os.path.join(dirname, filename)
#         df = pd.read_csv(filepath)
#         df = pre_process_data(df,60)
#         column = "Next Day Close Price GR"
#         (df,column) = dependent_column(df,column)
#         X = df.drop(columns=[column])
#         Y = df[column]
        
#         resultdf = pd.DataFrame(result)
#         resultdf.to_csv(os.path.join(os.getcwd(),str(filename[2:8])+"_gnn"+".csv"),index=None)

In [25]:
for name,table in tables.items():
    print(table)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                      GNN_Regression                                                                     |
+--------------------------------------------+------------------------------+----------+----------+----------+--------------+-----+----------+------------+
|                  Company                   |            Method            |   RMSE   |   MAE    |   MSE    | rsquared_adj | std | best-std | Percentage |
+--------------------------------------------+------------------------------+----------+----------+----------+--------------+-----+----------+------------+
| 500085-CHAMBAL FERTILISERS & CHEMICALS LTD |     ForwardSelection-0.0     | 0.030287 | 0.020414 | 0.000917 |   -0.00017   | 0.0 |   0.97   |  0.513447  |
| 500085-CHAMBAL FERTILISERS & CHEMICALS LTD |     ForwardSelect

In [26]:
final_df

,Company,Model,Method,Percentage
0,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,KNN-Regression,KNN_Regression_FI_BackwardElimination_,0.540342
192,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,GNN-Regression,GNN_Regression_FI_AllFeaturesConsideration_std...,0.526895
1,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,Ridge Regression,RidgeFIFValue1,0.506112
2,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,Linear Regression,LinearFIBackwardElimination,0.501222
3,500085-CHAMBAL FERTILISERS & CHEMICALS LTD,Lasso Regression,LassoFIFValue10,0.497555
...,...,...,...,...
187,542602-EMBASSY OFFICE PARKS REIT,Elastic Net Regression,ElasticFIPValue0.1,0.569231
188,542602-EMBASSY OFFICE PARKS REIT,Linear Regression,LinearFICoefficients0.1,0.538462
189,542602-EMBASSY OFFICE PARKS REIT,Ridge Regression,RidgeFICoefficients0.1,0.538462
190,542602-EMBASSY OFFICE PARKS REIT,KNN-Regression,KNN_Regression_FI_AllFeaturesConsideration_,0.530769
